# Mixture of Linear Regressions

In this note, we will solve the mixture of linear regressions problem using the moment method.

This block contains some $\LaTeX$ macros.
$\newcommand{\E}{\mathbb{E}}$
$\renewcommand{\Re}{\mathbb{R}}$
$\newcommand{\oft}[1]{{^{(#1)}}}$
$\newcommand{\oftt}[2]{{^{(#1)#2}}}$

In [32]:
import util
from util import *
from models import LinearRegressionsMixture
import numpy as np
np.set_printoptions(precision=3, suppress=True)
from IPython.display import display, Markdown, Math
from operator import mul
import sympy as sp
sp.init_printing()

Let $w_1, ..., w_K \in \Re^D$ be a set of $K$ regression coefficients. Let $x_1, ..., x_N \in \Re^D$ be a given set of data points and let $y_n = \sum_{k} \delta_k w_k^\top x_n + \epsilon$ be observed responses. Our objective is to recover $(w_k)$.

## Toy Example

Let's construct a toy example for the rest of this document.

In [33]:
K, D = 2, 2
np.random.seed(0)
pis = np.array([0.4, 0.6])
ws = np.array([[0.75, 0.25], [0.4, 0.9]])
model = LinearRegressionsMixture.generate("tmp.dat", K, D, betas = ws, weights = pis, cov = 1*np.eye(D))
print("True parameters:")
print(model.weights)
print(model.betas)

True parameters:
[ 0.4  0.6]
[[ 0.75  0.25]
 [ 0.4   0.9 ]]


# Noiseless, Infinite Data Setting

In this scenario, assume that $\epsilon = 0$ and that we observe expected moments of the data. Consider the moments:
\begin{align}
\E[x^\alpha y^b] 
&= \sum_k \pi_k \E[x^\alpha y^b | \delta_k = 1] \\
&= \sum_k \pi_k \E[x^\alpha [w_k^\top x]^b] \\
&= \sum_k \pi_k \E[x^\alpha \sum_{\beta \in p(b)} w_k^\beta x^\beta] \\
&= \sum_{\beta \in p(b)} \E_\pi[w^\beta] \E[x^{\alpha + \beta}],
\end{align}
where $p(b)$ are the $d$-partitions of $b$. Note that $\E[x^{\alpha + \beta}]$ are observable quantities, and thus this simply represents a mixture over the polynomials with terms $w_k^\beta$.

In [34]:
def evaluate_mixture(ws, pis, beta):
    """
    Compute E_\pi[w^beta]
    """
    return sum(pi * util.monomial(w, beta) for w, pi in zip(ws.T, pis))
    
def compute_exact_y_moments(ws, pis, moments_x, alpha, b):
    """
    Compute the exact moments E[x^a y^b] using coefficients ws, pis and moments_x.
    """
    D, K = ws.shape
    ret = 0.
    for beta in partitions(D, b):
        ret += evaluate_mixture(ws, pis, beta) * moments_x[tuple_add(alpha, beta)]
    return ret

def describe_moment_polynomial(R, moments_x, moment_y, alpha, b):
    """
    Computes the moment polynomial for E[x^alpha, y^b]
    """
    D = len(R.symbols)
    w = R.symbols
    expr = -moment_y
    for beta in partitions(D, b):
        expr += util.monomial(w, beta) * moments_x[tuple_add(alpha, beta)]
    return expr

In [35]:
# Example 
def double_factorial(n): 
    return reduce(mul, xrange(n, 0, -2)) if n > 0 else 1
def gaussian_moments(sigma, d):
    """
    E[x^d] where x is standard gaussian with sigma
    """
    if d == 0: return 1
    elif d % 2 == 0: return double_factorial(d-1) * sigma**d
    else: return 0
def expected_gaussian_moments(sigma, alphas):
    return {alpha : prod(gaussian_moments(sigma, a) for a in alpha) for alpha in alphas}
def expected_uniform_moments(alphas):
    return {alpha : 1.}
def expected_moments_y(ws, pis, moments_x, alphabs):
    return {(alpha, b) : compute_exact_y_moments(ws, pis, moments_x, alpha, b) for alpha, b in alphabs}

In [36]:
R, _ = sp.xring(['w%d'%d for d in xrange(D)], sp.RR, sp.grevlex)

deg_b, deg_x = 3, 3
sigma = 1.0
alphas = list(dominated_elements((deg_x for _ in xrange(D))))
alphabs = [(alpha, b) for alpha in alphas for b in xrange(1,deg_b+1)]
alphas_ = list(dominated_elements((deg_x + deg_b for _ in xrange(D))))
moments_x = expected_gaussian_moments(sigma, alphas_)
moments_y = expected_moments_y(ws, pis, moments_x, alphabs)
#display(moments)

In [37]:
def get_constraint_polynomials(moments_y, moments_x, deg_x, deg_b):
    constrs = []
    for b in xrange(1, deg_b+1):
        for alpha in util.dominated_elements((deg_x for _ in xrange(D))):
            constrs.append( describe_moment_polynomial(R, moments_x, moments_y[(alpha, b)], alpha, b) )
    return constrs

With this machinery, we can compute the moment polynomials required for the moment method magic!

In [38]:
from mompy.core import MomentMatrix
import mompy.solvers as solvers; reload(solvers)
import mompy.extractors as extractors; reload(extractors)

constrs_exact = get_constraint_polynomials(moments_y, moments_x, deg_x, deg_b)
M = MomentMatrix(3, R.symbols, morder='grevlex')
sol = solvers.solve_generalized_mom_coneqp(M, constrs_exact, None)
#sol = solvers.solve_basic_constraints(M, constrs_exact, slack=0)



     pcost       dcost       gap    pres   dres
 0:  1.2381e-01 -6.6245e-01  4e+01  5e+00  9e+00
 1:  1.2372e-01 -3.5044e+00  8e+00  8e-01  2e+00
 2:  1.2600e-01 -1.2251e+00  2e+00  1e-01  3e-01
 3:  3.0668e-02 -1.2356e-01  2e-01  8e-03  2e-02
 4:  4.6824e-03 -2.0966e-02  3e-02  1e-03  2e-03
 5:  8.3297e-04 -2.7415e-03  4e-03  1e-04  2e-04
 6:  2.5900e-04 -5.1225e-05  3e-04  9e-15  3e-16
 7:  6.1469e-05  7.7121e-06  5e-05  4e-15  2e-17
 8:  4.0140e-05  2.4754e-05  2e-05  2e-15  1e-17
 9:  2.9425e-05  2.8129e-05  1e-06  6e-15  5e-18
10:  2.8677e-05  2.8513e-05  2e-07  1e-15  2e-18
11:  2.8573e-05  2.8554e-05  2e-08  2e-15  4e-19
Optimal solution found.


In [39]:
display( model.betas)
display(extractors.extract_solutions_lasserre(M, sol['x'], Kmax=2))
display(extractors.extract_solutions_dreesen_proto(M, sol['x'], Kmax=2))


array([[ 0.75,  0.25],
       [ 0.4 ,  0.9 ]])

lost 0.0003014


the next biggest eigenvalue we are losing is 0.000301


## With real samples

Let's try to solve the problem with generated samples.


In [40]:
def compute_expected_moments(xs, alphas):
    moments = {alpha : 0. for alpha in alphas}
    for i, x in enumerate(xs):
        for alpha in alphas:
            moments[alpha] += (monomial(x, alpha) - moments[alpha])/(i+1)
    return moments

def compute_expected_moments_y(ys, xs, alphabs):
    moments = {(alpha, b) : 0. for alpha, b in alphabs}
    for i, (y, x) in enumerate(zip(ys,xs)):
        for alpha, b in alphabs:
            moments[(alpha,b)] += (monomial(x, alpha) * y**b - moments[(alpha,b)])/(i+1)
    return moments

In [41]:
ys, xs = model.sample(1e3)
moments_x = compute_expected_moments(xs, alphas_)
moments_y = compute_expected_moments_y(ys, xs, alphabs)

In [42]:

from mompy.core import MomentMatrix
import mompy.solvers as solvers; reload(solvers)
import mompy.extractors as extractors; reload(extractors)

constrs_noisy = get_constraint_polynomials(moments_y, moments_x, deg_x, deg_b)
#display(constrs_noisy)
M = MomentMatrix(2, R.symbols, morder='grevlex')
solsdp = solvers.solve_generalized_mom_coneqp(M, constrs_noisy, None)
#solsdp = solvers.solve_basic_constraints(M, constrs_noisy, slack=2e-2)
#sol = solvers.solve_moments_with_convexiterations(M, constrs, 3)
print solsdp['x']

     pcost       dcost       gap    pres   dres
 0:  5.2788e-01  1.1514e+00  3e+01  4e+00  9e+00
 1:  6.1636e-01 -2.2758e+00  6e+00  8e-01  2e+00
 2:  8.0989e-01  2.6549e-01  1e+00  1e-01  3e-01
 3:  1.0449e+00  9.1622e-01  2e-01  3e-02  6e-02
 4:  9.6847e-01  8.3137e-01  2e-01  2e-02  5e-02
 5:  1.0159e+00  9.7422e-01  6e-02  5e-03  1e-02
 6:  9.7796e-01  9.3639e-01  5e-02  4e-03  8e-03
 7:  9.8520e-01  9.7132e-01  2e-02  1e-03  2e-03
 8:  9.7040e-01  9.5611e-01  2e-02  9e-04  2e-03
 9:  9.7176e-01  9.6524e-01  7e-03  3e-04  6e-04
10:  9.6446e-01  9.5761e-01  8e-03  2e-04  5e-04
11:  9.6504e-01  9.6174e-01  4e-03  7e-05  1e-04
12:  9.6154e-01  9.5812e-01  4e-03  6e-05  1e-04
13:  9.6166e-01  9.6017e-01  2e-03  2e-05  3e-05
14:  9.6087e-01  9.5941e-01  1e-03  1e-05  3e-05
15:  9.6080e-01  9.6032e-01  5e-04  3e-06  6e-06
16:  9.6065e-01  9.6027e-01  4e-04  2e-06  4e-06
17:  9.6057e-01  9.6055e-01  2e-05  8e-08  2e-07
18:  9.6056e-01  9.6056e-01  2e-06  2e-09  3e-09
19:  9.6056e-01  9.60

In [43]:
display(model.betas)
display(extractors.extract_solutions_lasserre(M, solsdp['x'], Kmax=2))

display(extractors.extract_solutions_dreesen_proto(M, solsdp['x'], Kmax=2))
sol = extractors.extract_solutions_dreesen_proto(M, solsdp['x'], Kmax=2)
ws_rec = array([sol[sp.symbols('w0')], sol[sp.symbols('w1')]])

array([[ 0.75,  0.25],
       [ 0.4 ,  0.9 ]])

the next biggest eigenvalue we are losing is 2.347730


the next biggest eigenvalue we are losing is 2.347730


In [44]:
for i in xrange(10):
    print constrs_exact[i];
    print constrs_noisy[i];
    print '\n'

0
0.0188708423854122*w0 + 0.325169089536888*w1 - 0.655822620203579


3.0*w0 - 1.35
3.71422540050967*w0 + 0.182057946395849*w1 - 1.65141794160749


0
0.132891465301014*w0 + 0.167703665140313*w1 - 0.259104490439454


3.0*w0 - 1.35
3.0756322241925*w0 + 0.0667059804850347*w1 - 1.42752983532732


3.0*w1 - 2.1
0.182057946395849*w0 + 3.18537672648181*w1 - 2.53678282781772


0
0.167703665140313*w0 + 0.0746383200434478*w1 - 0.0430938784994356


1.0*w1 - 0.7
0.0667059804850347*w0 + 1.08667330449701*w1 - 0.82711044922914


0
0.185532531523686*w0 + 0.0377319657154177*w1 - 0.0728439621793891


0
0.0746383200434478*w0 - 0.0697794616377764*w1 - 0.0420840656232283


1.0*w0 - 0.45
1.08667330449701*w0 + 0.0104407651840518*w1 - 0.471931764725507




In [45]:
M.numeric_instance(solsdp['x'])

array([[   1.   ,    0.469,    0.723,    0.328,    0.473,    0.69 ],
       [   0.469,    0.328,    0.473,    0.116,    0.374,    0.612],
       [   0.723,    0.473,    0.69 ,    0.374,    0.612,    0.574],
       [   0.328,    0.116,    0.374,  105.501,   16.951,    7.809],
       [   0.473,    0.374,    0.612,   16.951,    7.809,  -25.251],
       [   0.69 ,    0.612,    0.574,    7.809,  -25.251,  160.14 ]])

In [46]:
import scipy as sc
import numpy.linalg
np.linalg.matrix_rank((M.numeric_instance(solsdp['x'])), 1e-2)

In [47]:
M.row_monos

In [48]:
M.numeric_instance(solsdp['x'])

array([[   1.   ,    0.469,    0.723,    0.328,    0.473,    0.69 ],
       [   0.469,    0.328,    0.473,    0.116,    0.374,    0.612],
       [   0.723,    0.473,    0.69 ,    0.374,    0.612,    0.574],
       [   0.328,    0.116,    0.374,  105.501,   16.951,    7.809],
       [   0.473,    0.374,    0.612,   16.951,    7.809,  -25.251],
       [   0.69 ,    0.612,    0.574,    7.809,  -25.251,  160.14 ]])

In [49]:
moments_y

In [ ]:
moments_x_true = expected_gaussian_moments(sigma, alphas_)
moments_y_true = expected_moments_y(ws, pis, moments_x, alphabs)

In [ ]:
for key in moments_y:
    display(key, abs(moments_y[key] - moments_y_true[key]))